In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
import pickle
import numpy as np
# Rends nos tracés inline
%matplotlib inline

In [2]:
df=pd.read_csv('corpus.csv')
df.head()

date            user  rating  \
0       Commenté en France le 9 août 2018         Justine     1.0   
1      Commenté en France le 21 juin 2018     Gouyer Eric     1.0   
2  Commenté en France le 13 décembre 2018      Peineau A.     5.0   
3   Commenté en France le 23 juillet 2018  Solenn Laurent     1.0   
4    Commenté en France le 1 janvier 2019             val     1.0   

                                              review  
0  Le téléphone n a jamais fonctionné et Apple no...  
1  Soit-disant le téléphone est certifié recondit...  
2  Reçu aujourd'hui via UPS standard, bien emball...  
3  iPhone reçu, la batterie ne tient pas une jour...  
4  J’ai offert ce portable à mon fils pour Noël.....

In [3]:
len(df),len(df.drop_duplicates('review'))

(1347, 1347)

In [4]:
df['l_review']=df['review'].apply(lambda x:len(x.split(' ')))
df

date                user  rating  \
0           Commenté en France le 9 août 2018             Justine     1.0   
1          Commenté en France le 21 juin 2018         Gouyer Eric     1.0   
2      Commenté en France le 13 décembre 2018          Peineau A.     5.0   
3       Commenté en France le 23 juillet 2018      Solenn Laurent     1.0   
4        Commenté en France le 1 janvier 2019                 val     1.0   
...                                       ...                 ...     ...   
1342       Commenté en France le 16 août 2020               Kasei     3.0   
1343     Commenté en France le 2 juillet 2020          Josué KAZI     4.0   
1344      Commenté en France le 21 avril 2020  Familyismyreligion     5.0   
1345  Commenté en France le 21 septembre 2019             charret     3.0   
1346       Commenté en France le 26 août 2020               Fatna     3.0   

                                                 review  l_review  
0     Le téléphone n a jamais fonctionné et Apple no...        13  
1     Soit-disant le téléphone est certifié recondit...        39  
2     Reçu aujourd'hui via UPS standard, bien emball...        58  
3     iPhone reçu, la batterie ne tient pas une jour...        59  
4     J’ai offert ce portable à mon fils pour Noël.....        79  
...                                                 ...       ...  
1342  au bout d'une semaine, le téléphone m'indique ...        22  
1343  Produit fiable.J’ai reçu le téléphone depuis u...        23  
1344  Livraison très rapide, plus rapide que prévu e...        30  
1345                la batterie se décharge très vite .         7  
1346  Bonjour le chargeur était défectueux et ne fon...        22  

[1347 rows x 5 columns]

In [5]:
df[(df['rating']<3) & (df['l_review']>5)].describe()

rating    l_review
count  391.000000  391.000000
mean     1.214834   33.629156
std      0.411233   27.865852
min      1.000000    6.000000
25%      1.000000   14.000000
50%      1.000000   26.000000
75%      1.000000   44.000000
max      2.000000  218.000000

In [6]:
len(df[(df['rating']>3) & (df['l_review']>5)]),len(df[(df['rating']<3)  & (df['l_review']>5)])

(582, 391)

In [7]:
df=df[df['l_review']>5]

In [8]:
len(df[df['rating']>3]),len(df[df['rating']<3])

(582, 391)

In [9]:
df['label']=df['rating']

# Je conserve autant de comments par label que possible (contraint equilibrer les classes 0 et 1)

positif=df[df['label']>3].sample(391)
#positif.index=list(range(0,len(positif)))

negatif=df[df['label']<3]
#negatif.index=list(range(len(positif),len(negatif)+len(positif)))

Corpus=pd.concat([positif,negatif],ignore_index=True)[['review','label']]

C:\Users\valen\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
Corpus

review  label
0    Le prix pour cet iPhone était imbattable, mis ...    5.0
1    iPhone avec suffisamment de mémoire , très bon...    5.0
2    Telephone livré avec un écouteur équipé d’une ...    4.0
3    Je l'ai acheté pour ma fille, elle le trouve s...    5.0
4    Rien à dire sur le produit très bon rapport qu...    5.0
..                                                 ...    ...
777  Article reçu en très bon état, sauf la batteri...    1.0
778  le téléphone reçu est un 32 go au lieu des 128...    1.0
779  Téléphone en bon état, les écouteurs sont non ...    2.0
780                    La batrie ce decharge trop vite    1.0
781  Je suis assez mécontente !!La batterie affiche...    2.0

[782 rows x 2 columns]

In [11]:
### Attention c'est sale, si vous avez un plus gros corpus travaillez avec df.loc[liste_index,'nom_colonne']
for ind in Corpus['label'].index:
    if Corpus.loc[ind,'label'] > 3:
        Corpus.loc[ind,'label']=1
    elif Corpus.loc[ind,'label'] < 3:
        Corpus.loc[ind,'label']=0

In [12]:
Corpus.label.value_counts()

0.0    391
1.0    391
Name: label, dtype: int64

In [13]:
# Manipulation du fench stemmer

from nltk.stem import SnowballStemmer

fr = SnowballStemmer('french')


In [14]:
# Construction de la liste des stop words french
from nltk.corpus import stopwords
from stop_words import get_stop_words
import nltk

nltk.download('stopwords')
my_stop_word_list = get_stop_words('french')
final_stopwords_list = stopwords.words('french')

s_w=list(set(final_stopwords_list+my_stop_word_list))
s_w=[elem.lower() for elem in s_w]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\valen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
!pip install stop-words

In [16]:
s_w

['eussions',
 'eûtes',
 'elle',
 'avant',
 'devriez',
 'droite',
 'faites',
 'es',
 'aie',
 'eûmes',
 'ceux',
 'serions',
 'faisez',
 'quelles',
 'auriez',
 'soient',
 'eussiez',
 'j',
 'parce',
 'avions',
 'auras',
 'ou',
 'aux',
 'début',
 'même',
 'leurs',
 'soyez',
 'vos',
 'au',
 'quel',
 'fussiez',
 'sois',
 'se',
 'la',
 'à',
 'sous',
 'a',
 'devrait',
 'avez',
 'seriez',
 'aurai',
 'seulement',
 'quelle',
 'pas',
 'un',
 'tels',
 'force',
 'ayantes',
 'sur',
 'serons',
 'étions',
 'soit',
 'car',
 'cela',
 'peut',
 'nommé',
 'étiez',
 'eussent',
 'voient',
 'mes',
 'aurez',
 'eue',
 'nous',
 'sera',
 'doit',
 'serai',
 'seraient',
 'sujet',
 'le',
 'trop',
 'vu',
 'cet',
 'aient',
 'moi',
 'eut',
 'ton',
 'tes',
 'ait',
 'auraient',
 'depuis',
 'avait',
 'ici',
 'nouveaux',
 'eux',
 'nommée',
 'dois',
 'on',
 's',
 'en',
 'qui',
 'eût',
 'dedans',
 'juste',
 'quand',
 'il',
 'et',
 'étants',
 'fussent',
 'dos',
 'alors',
 'était',
 'dès',
 'ont',
 'état',
 'moins',
 'vous',
 'm

In [17]:
import re
from unidecode import unidecode

def nettoyage(string):
    l=[]
    string=unidecode(string.lower())
    #Sans ponctuation pour le moment
    string=" ".join(re.findall("[a-zA-Z]+", string))
    
    for word in string.split():
        #if word in s_w:
        #    continue
        #else:
            l.append(fr.stem(word))
    return ' '.join(l)

nettoyage(Corpus['review'].loc[0])

'le prix pour cet iphon etait imbatt mis a part un leg coup sur la coqu arrier que j ai facil cach a l aid d une coqu'

In [18]:
!pip install Unidecode

In [19]:
Corpus['review_net']=Corpus['review'].apply(nettoyage)

In [20]:


vectorizer = TfidfVectorizer()
vectorizer.fit(Corpus['review_net'])
X=vectorizer.transform(Corpus['review_net'])
print(vectorizer.get_feature_names())

#Save vectorizer.vocabulary_
pickle.dump(vectorizer.vocabulary_,open("feature.pkl","wb"))




['abandon', 'abim', 'abime', 'abord', 'about', 'absenc', 'absent', 'absolu', 'absten', 'abus', 'acce', 'accept', 'accessoir', 'accompagn', 'accord', 'accus', 'achat', 'achatrecu', 'achet', 'acheteur', 'acqui', 'acquisit', 'actif', 'activ', 'activit', 'actuel', 'adapt', 'adher', 'admet', 'ado', 'ador', 'adress', 'affair', 'affich', 'affichag', 'afin', 'agir', 'agiss', 'agit', 'agre', 'agreabl', 'ai', 'aid', 'aient', 'ail', 'ailleur', 'aim', 'ains', 'air', 'ait', 'ajour', 'aliment', 'allais', 'allait', 'alle', 'aller', 'allez', 'allum', 'allumag', 'alor', 'alum', 'amazon', 'amen', 'americain', 'amont', 'an', 'ancien', 'and', 'angle', 'anglej', 'anne', 'anniv', 'anniversair', 'annonc', 'annonce', 'anomal', 'anormal', 'ansje', 'anten', 'anti', 'apar', 'apart', 'apercois', 'apercus', 'appar', 'apparaissent', 'apparei', 'appareil', 'appareilest', 'apparent', 'appart', 'apparu', 'appel', 'apple', 'applem', 'appli', 'appliqu', 'apporte', 'apprec', 'appris', 'appui', 'apre', 'arc', 'argent', 'a

In [21]:
# Taille du vocabulaire (mots unique)
print(len(vectorizer.get_feature_names()))

1884


In [22]:
y=Corpus['label']
y

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
777    0.0
778    0.0
779    0.0
780    0.0
781    0.0
Name: label, Length: 782, dtype: float64

In [23]:
X

<782x1884 sparse matrix of type '<class 'numpy.float64'>'
	with 17025 stored elements in Compressed Sparse Row format>

In [24]:
X.shape

(782, 1884)

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [26]:
from sklearn.linear_model import LogisticRegression
cls=LogisticRegression(max_iter=300).fit(x_train,y_train)

print(cls.score(x_val,y_val))


#Save classifier
pickle.dump(cls,open("cls.pkl","wb"))

0.8471337579617835


In [27]:
phrase='Je ne suis vraiment pas content, c est chiant'

user=vectorizer.transform([nettoyage(phrase)])

cls.predict_proba(user), cls.predict(user)




(array([[0.47396227, 0.52603773]]), array([1.]))

In [28]:
phrase='Je suis vraiment content, c est parfait'

user=vectorizer.transform([nettoyage(phrase)])

cls.predict_proba(user), cls.predict(user)




(array([[0.16326939, 0.83673061]]), array([1.]))

In [29]:
phrase='mauvais'

user=vectorizer.transform([nettoyage(phrase)])

cls.predict_proba(user), cls.predict(user)




(array([[0.56830344, 0.43169656]]), array([0.]))

In [30]:
phrase="nul a chier a la mort c'est nul, merde pas article !"


#Load 
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
user = transformer.fit_transform(loaded_vec.fit_transform([nettoyage(phrase)]))

cls=pickle.load(open("cls.pkl", "rb"))

cls.predict(user),cls.predict_proba(user).max(),

print(cls.predict(user))
if cls.predict(user) >= 1:
    print("Super tu apprécies ")
else :
    print("Ok bad trip ")


[0.]
Ok bad trip 


In [31]:
import requests

route='/prediction'
url='http://127.0.0.1:5000'+route
phrase="nul a chier a la mort c'est nul, merde pas article !"
param={'input_text': phrase}
r=requests.post(url,data=param)
r.status_code

500

In [32]:
X

<782x1884 sparse matrix of type '<class 'numpy.float64'>'
	with 17025 stored elements in Compressed Sparse Row format>

In [33]:
X.shape

(782, 1884)

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [35]:
from sklearn.linear_model import LogisticRegression
cls=LogisticRegression(max_iter=300).fit(x_train,y_train)

print(cls.score(x_val,y_val))


#Save classifier
pickle.dump(cls,open("cls.pkl","wb"))

0.8853503184713376


In [36]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
import pickle
import numpy as np
import re
from unidecode import unidecode
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from stop_words import get_stop_words
from sklearn.linear_model import LogisticRegression
# Rends nos tracés inline
%matplotlib inline
df=pd.read_csv('corpus.csv')
len(df),len(df.drop_duplicates('review'))
df['l_review']=df['review'].apply(lambda x:len(x.split(' ')))
df[(df['rating']<3) & (df['l_review']>5)].describe()
len(df[(df['rating']>3) & (df['l_review']>5)]),len(df[(df['rating']<3)  & (df['l_review']>5)])
df=df[df['l_review']>5]
len(df[df['rating']>3]),len(df[df['rating']<3])
df['label']=df['rating']

# Je conserve autant de comments par label que possible (contraint equilibrer les classes 0 et 1)

positif=df[df['label']>3].sample(391)
#positif.index=list(range(0,len(positif)))

negatif=df[df['label']<3]
#negatif.index=list(range(len(positif),len(negatif)+len(positif)))

Corpus=pd.concat([positif,negatif],ignore_index=True)[['review','label']]
for ind in Corpus['label'].index:
    if Corpus.loc[ind,'label'] > 3:
        Corpus.loc[ind,'label']=1
    elif Corpus.loc[ind,'label'] < 3:
        Corpus.loc[ind,'label']=0
        
Corpus.label.value_counts()

fr = SnowballStemmer('french')
my_stop_word_list = get_stop_words('french')
final_stopwords_list = stopwords.words('french')

s_w=list(set(final_stopwords_list+my_stop_word_list))
s_w=[elem.lower() for elem in s_w]

def nettoyage(string):
    l=[]
    string=unidecode(string.lower())
    #Sans ponctuation pour le moment
    string=" ".join(re.findall("[a-zA-Z]+", string))
    
    for word in string.split():
        #if word in s_w:
        #    continue
        #else:
            l.append(fr.stem(word))
    return ' '.join(l)

nettoyage(Corpus['review'].loc[1])
Corpus['review_net']=Corpus['review'].apply(nettoyage)

vectorizer = TfidfVectorizer()
vectorizer.fit(Corpus['review_net'])
X=vectorizer.transform(Corpus['review_net'])

#Save vectorizer.vocabulary_
pickle.dump(vectorizer.vocabulary_,open("feature.pkl","wb"))
print(len(vectorizer.get_feature_names()))
y=Corpus['label']
X.shape
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)
cls=LogisticRegression(max_iter=300).fit(x_train,y_train)

print(cls.score(x_val,y_val))


#Save classifier
pickle.dump(cls,open("cls.pkl","wb"))


1897
0.8535031847133758
